In [1]:
import backtrader as bt
from main_module import main
import sys
import os
from datetime import datetime
import yfinance as yf
from indicators import relativestrength
from indicators import supertrend
import numpy as np
from main_module import main
from main_module import config
import plotly.graph_objects as go


In [2]:
args = config.Config("config.yaml")
df = main.capitalize(args)

In [7]:
foo_filename = "./foo.png"
if os.path.exists(foo_filename):
    os.remove(foo_filename)

df = yf.download("AAPL", start="2023-01-01", end="2023-11-09", interval="1h")

close = df["Close"].to_numpy()
rsi, indicator, long, short, sma = relativestrength.relative_strength_index(close)
entry, exit, roi, earning = supertrend.backtest(
        close,
        indicator,
        investment=10000,
        debug=False,
        commission=0,
    )

print(f"Earning from investing ${10000} is ${round(earning,2)} (ROI = {roi}%)")


[*********************100%%**********************]  1 of 1 completed
Earning from investing $10000 is $4541.55 (ROI = 45.42%)


In [21]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close']),
                go.Scatter(x=df.index, y=long, mode="markers", marker=dict(symbol="arrow",size=15),),
                go.Scatter(x=df.index, y=short, mode="markers")
                ])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=[17, 9], pattern="hour"), #hide hours outside of 9am-5pm
    ]
)
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "sun"]) #hide weekends
        #,dict(values=["2015-12-25", "2023-01-01"])  # hide Christmas and New Year's
    ]
)
fig.show()

fig = go.Figure([go.Scatter(x=df.index, y=rsi), go.Scatter(x=df.index, y=sma)])
fig.add_hline(y=70, line=dict(color='Green'))
fig.add_hline(y=30, line=dict(color='Red'))
#fig.update_xaxes(
#    rangebreaks=[
#        dict(bounds=["sat", "mon"]), #hide weekends
#        dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
#    ]
#)
fig.update_xaxes(
    rangebreaks=[
        dict(bounds=[17, 9], pattern="hour"), #hide hours outside of 9am-5pm
    ]
)
fig.show()